In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=3)
search.invoke("Who are the top stars of the 2024 Eurocup?")

[{'url': 'https://www.nbcsports.com/soccer/news/euro-2024-player-power-rankings-who-are-the-top-20-stars',
  'content': "EURO 2024 player Power Rankings. 20. Giorgi Mamardashvili (Georgia) 19. Khvicha Kvaratskhelia (Georgia) 18. Hakan Calhanoglu (Turkey) 17. Lamine Yamal (Spain) 16. N'Golo Kante (France) 15. Jude Bellingham (England) 14."},
 {'url': 'https://www.givemesport.com/best-players-at-euro-2024-ranked/',
  'content': "11 20-11 Stars like Toni Kroos and Cristiano Ronaldo feature. While they don't quite break into the top 10 best players at Euro 2024, the talent from 11 to 20 is still pretty incredible."},
 {'url': 'https://www.givemesport.com/ranking-best-players-euro-2024/',
  'content': "France's Kylian Mbappe is ranked as the best player at Euro 2024. With the knockout stages of Euro 2024 about to get underway, the best that Europe has to offer will be battling it out to get"}]

In [6]:
tools = [search]

In [7]:
tools

[TavilySearchResults(max_results=3)]

In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [9]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="Where is the soccer Eurocup 2024 played?")]})

response["messages"]

[HumanMessage(content='Where is the soccer Eurocup 2024 played?', id='800bb4ad-fdbc-492a-b751-20001c83e54d'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ekuusa5Fi6tGgbwmQNbVsskk', 'function': {'arguments': '{"query":"Soccer Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 92, 'total_tokens': 117, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b92b683f-7d38-4d34-bcc6-5e398c5b027f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Soccer Eurocup 2024 location'}, 'id': 'call_Ekuusa5Fi6tGgbwmQNbVsskk', 'type': 'tool_call'}], usage_metadata={'input_tok

In [10]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="What is Python?")]})

response["messages"]

[HumanMessage(content='What is Python?', id='8672de02-fe07-4744-905d-b9e0161d31e3'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CE6KtNSqdNcjpbzyqMsJ8Jxh', 'function': {'arguments': '{"query":"Python"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 85, 'total_tokens': 103, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-38592a6f-58c5-4a9d-aaa8-4173d9edfd12-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Python'}, 'id': 'call_CE6KtNSqdNcjpbzyqMsJ8Jxh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 85, 'output_tokens': 18, 'total_tokens': 103}),
 ToolMessage(co

In [11]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="When and where will it be the 2024 Eurocup final match?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1T8TQJZOBI9Tjwm9fogzi1NO', 'function': {'arguments': '{"query":"2024 Eurocup final match location and date"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 96, 'total_tokens': 122, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e7f3efff-5390-4da6-b5dd-067748278536-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Eurocup final match location and date'}, 'id': 'call_1T8TQJZOBI9Tjwm9fogzi1NO', 'type': 'tool_call'}], usage_metadata={'input_tokens': 96, 'output_tokens': 26, 'total_tokens': 122})]}}
---

## Adding memory
* Adding memory in LangGraph is very similar to what we did with LangChain.

In [12]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [13]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "001"}}

In [16]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who won the 2024 soccer Eurocup?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='I am sorry for the inconvenience. Let me find the information about the winner of the 2024 soccer Eurocup for you.', additional_kwargs={'tool_calls': [{'id': 'call_gb9W8DQ0zx0orw1nRwXDRYJy', 'function': {'arguments': '{"query":"2024 soccer Eurocup winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 929, 'total_tokens': 979, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ce696981-8dd4-4319-b62a-b0f2be73caa9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 soccer Eurocup winner'}, 'id': 'call_gb9W8DQ0zx0orw1nRwXDRYJy', 'type': 'tool_call'}], us

In [18]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who were the top stars of that winner team?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bI7uTHk2LrxI9B4kpdBIrBHh', 'function': {'arguments': '{"query":"top players of Spain national team Euro 2024"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 1764, 'total_tokens': 1791, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0efd7059-7feb-42aa-a933-993e716ee6cf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'top players of Spain national team Euro 2024'}, 'id': 'call_bI7uTHk2LrxI9B4kpdBIrBHh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1764, 'output_tokens': 27, 'total_tokens': 179

In [ ]:
config = {"configurable": {"thread_id": "002"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what soccer team we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='We were discussing the Spain national soccer team and their victory in the 2024 soccer Eurocup. If you have any more questions or need further information, feel free to ask!', response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 2141, 'total_tokens': 2178, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-03a75b3f-3017-4a6a-a39a-84f1060ffd0b-0', usage_metadata={'input_tokens': 2141, 'output_tokens': 37, 'total_tokens': 2178})]}}
----
